In [17]:
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
import torch
import gymnasium as gym
import gym_bart
from plotting_utils import (
    set_rc, 
    add_abc_to_subaxes, 
    colors, 
    rgb_colors
)
from model_evaluation import forced_action_evaluate as evalu

import sys
sys.path.append('../')
from ppo.envs import make_vec_env

In [18]:
model, (obs_rms, ret_rms) = torch.load('../saved_checkpoints/behav/behav_t0/50.pt')
envs = make_vec_env("BartEnv-v0")

In [21]:
envs.reset()
envs.step(torch.tensor([[1]]))

(array([[ 0.        ,  0.        ,  0.00499969,  0.        ,  0.        ,
          0.98022395, -0.99985003,  0.99995   ]], dtype=float32),
 array([0.], dtype=float32),
 array([False]),
 [{'current_color': 2,
   'last_size': 0.06996683054520714,
   'popped': False,
   'inflate_delay': 0,
   'balloon_limit': 0.7967095004639595,
   'TimeLimit.truncated': False}])

In [23]:
%run model_evaluation
evalu = forced_action_evaluate
res = evalu(model, obs_rms, env_name="BartEnv-v0")

In [25]:
res.keys()

dict_keys(['obs', 'actions', 'action_log_probs', 'action_probs', 'rewards', 'rnn_hxs', 'dones', 'masks', 'envs', 'data', 'activations', 'values', 'actor_features', 'auxiliary_preds', 'auxiliary_truths'])

In [12]:

import sys
sys.path.insert(0, '..')
from ppo.envs import make_vec_env

env_name = "BartEnv-v0"
num_processes = 1
seed = 0
env_kwargs = {}
auxiliary_tasks = []
auxiliary_task_args = []
normalize = True
device = torch.device("cpu")


envs = make_vec_env(env_name, 
                        seed=seed, 
                        n_envs=num_processes,
                        env_kwargs=env_kwargs,
                        auxiliary_tasks=auxiliary_tasks,
                        auxiliary_task_args=auxiliary_task_args,
                        normalize=normalize,
                        dummy=True,)

if normalize:
    envs.training = False
if obs_rms is not None:
    envs.obs_rms = obs_rms

eval_episode_rewards = []

all_obs = []
all_actions = []
all_action_log_probs = []
all_action_probs = []
all_rewards = []
all_rnn_hxs = []
all_dones = []
all_masks = []
all_activations = []
all_values = []
all_actor_features = []
all_auxiliary_preds = []
all_auxiliary_truths = []
data = {}

ep_obs = []
ep_actions = []
ep_action_log_probs = []
ep_action_probs = []
ep_rewards = []
ep_rnn_hxs = []
ep_dones = []
ep_values = []
ep_masks = []
ep_actor_features = []

ep_auxiliary_preds = []
ep_activations = []
ep_auxiliary_truths = []


obs = envs.reset()
rnn_hxs = torch.zeros(
    num_processes, model.recurrent_hidden_state_size, device=device)
masks = torch.zeros(num_processes, 1, device=device)

step = 0
ep_obs.append(obs)
ep_rnn_hxs.append(rnn_hxs)
with torch.no_grad():
    outputs = model.act(torch.tensor(obs), rnn_hxs, 
                            masks, deterministic=True,
                            with_activations=True)